In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("TunningCross").getOrCreate()

23/12/28 15:25:09 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/28 15:25:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 15:25:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/28 15:25:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
iris = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/iris.csv", header=True, inferSchema=True, sep=",")
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

In [3]:
from pyspark.ml.feature import VectorAssembler

vector = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")

In [4]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class", outputCol="label")

In [5]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlp=MultilayerPerceptronClassifier(maxIter=100, layers=[4,5,4,3], featuresCol="independente", labelCol="label")

In [6]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vector, indexer, mlp])

In [7]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

performance = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
grid = ParamGridBuilder().addGrid(mlp.maxIter, [10, 100, 1000]).addGrid(mlp.layers, [[4, 5, 4, 3], [4, 4, 3]]).build()
crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=grid, evaluator=performance, numFolds=3)

In [8]:
model = crossval.fit(irisTreino)

23/12/28 15:40:06 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/12/28 15:40:06 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
23/12/28 15:40:06 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/12/28 15:40:06 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
23/12/28 15:40:06 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.125
23/12/28 15:40:06 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.0625
23/12/28 15:40:06 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.03125
23/12/28 15:40:07 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.015625
23/12/28

In [10]:
previsao = model.transform(irisTeste)
previsao.select("rawprediction", "probability", "prediction").show()

+--------------------+--------------------+----------+
|       rawprediction|         probability|prediction|
+--------------------+--------------------+----------+
|[-83.723687483586...|[5.46600101908790...|       1.0|
|[-83.666410102685...|[6.27707708094085...|       1.0|
|[-83.671132724770...|[6.20586497704423...|       1.0|
|[-83.761903660500...|[4.98397893228488...|       1.0|
|[-83.699051832652...|[5.80113844940409...|       1.0|
|[-83.643337910371...|[6.63682307347145...|       1.0|
|[-83.630983148613...|[6.8378726551842E...|       1.0|
|[6.40884144671795...|[7.57469490094834...|       2.0|
|[26.4468880122728...|[0.92307203960881...|       0.0|
|[-83.680186713405...|[6.07161481306949...|       1.0|
|[0.97615641538063...|[4.53159908483809...|       2.0|
|[-83.419493553190...|[1.13967241465815...|       1.0|
|[6.93247986145824...|[1.24063391176264...|       2.0|
|[-83.536774850361...|[8.58514727513422...|       1.0|
|[-83.609527536741...|[7.20157532112932...|       1.0|
|[10.82006

In [11]:
performance = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(f"Acuracia: {acuracia}")

Acuracia: 0.9545454545454546
